# **변형된 문제를 가지고 결괏값을 검증하자.**


3개운송사_3개운송구간_기준정보

4개운송사_3개운송구간_기준정보

1개운송사만선정_기준정보

In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 96.4 MB/s eta 0:00:00


3개운송사_3개운송구간_기준정보

In [2]:
import pandas as pd
import time
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger, LpBinary, value
import math

def optimize_truck_assignment():
    start_time = time.time()

    #--------------------------------------------------------
    # 1) 엑셀 데이터 읽기
    #--------------------------------------------------------
    input_file = "/content/sample_data/3개운송사_3개운송구간_기준정보.xlsx"

    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']

    max_trucker_num = pd.read_excel(input_file, sheet_name=0, usecols="M").iloc[0, 0]
    max_ratio = pd.read_excel(input_file, sheet_name=0, usecols="P").iloc[0, 0]

    # 데이터 클리닝
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)
    df_demand.dropna(subset=['필요대수'], inplace=True)

    #--------------------------------------------------------
    # 2) 파라미터 구성
    #--------------------------------------------------------
    cost_dict = {}
    capacity_dict = {}
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])
        cost_dict[key] = row['운송단가']
        capacity_dict[key] = row['트럭대수']

    demand_dict = {}
    for _, row in df_demand.iterrows():
        key = (row['운송구간'], row['트럭유형'])
        demand_dict[key] = row['필요대수']

    truckers = df['운송사'].unique()
    route_type_list = demand_dict.keys()

    #--------------------------------------------------------
    # 3) 모델 정의
    #--------------------------------------------------------
    prob = LpProblem("Truck_Optimization", LpMinimize)

    x_vars = {}
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = LpVariable(f"x_{i}_{r}_{t}", lowBound=0, cat=LpInteger)

    y_vars = {i: LpVariable(f"y_{i}", cat=LpBinary) for i in truckers}

    #--------------------------------------------------------
    # 4) 목적함수
    #--------------------------------------------------------
    prob += lpSum([cost_dict[(i, r, t)] * x_vars[(i, r, t)] for (i, r, t) in x_vars]), "Total_Transport_Cost"

    #--------------------------------------------------------
    # 5) 제약조건
    #--------------------------------------------------------

    # (1) 수요 충족
    for (r, t) in route_type_list:
        prob += lpSum([x_vars[(i, r, t)] for i in truckers if (i, r, t) in x_vars]) == demand_dict[(r, t)], f"Demand_{r}_{t}"

    # (2) 운송사별 보유 트럭수 제한
    for (i, r, t), cap in capacity_dict.items():
        prob += x_vars[(i, r, t)] <= cap, f"Capacity_{i}_{r}_{t}"

    # (3) 선택 운송사만 배정
    for i in truckers:
        for (r, t) in route_type_list:
            if (i, r, t) in x_vars:
                prob += x_vars[(i, r, t)] <= capacity_dict[(i, r, t)] * y_vars[i], f"Link_Select_{i}_{r}_{t}"

    # (4) 최대 선택 운송사 수 제한
    prob += lpSum([y_vars[i] for i in truckers]) <= max_trucker_num, "Max_Selected_Truckers"

    # (5) 트럭유형-운송구간별 운송사 배정 상한 제한
    for (r, t) in route_type_list:
        demand = demand_dict[(r, t)]
        x_max = math.ceil(demand * max_ratio)
        for i in truckers:
            if (i, r, t) in x_vars:
                prob += x_vars[(i, r, t)] <= x_max, f"MaxRatio_{i}_{r}_{t}"

    #--------------------------------------------------------
    # 6) 최적화 수행
    #--------------------------------------------------------
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    status = pulp.LpStatus[prob.status]

    print("Status:", status)
    if prob.status not in [1, 2]:
        print("유효한 해를 찾지 못했습니다.")
        return

    total_cost = value(prob.objective)
    print("총 운송비:", total_cost)

    #--------------------------------------------------------
    # 7) 결과 저장
    #--------------------------------------------------------
    result_data = []
    for (i, r, t), var in x_vars.items():
        if var.varValue > 0:
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(var.varValue)])

    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    df_result.sort_values(by=['운송구간', '트럭유형'], inplace=True)

    output_file = "/content/sample_data/3개운송사_3개운송구간_최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))

    print("\n프로그램 수행시간(초):", round(time.time() - start_time, 4))


if __name__ == "__main__":
    optimize_truck_assignment()



Status: Optimal
총 운송비: 32200.0

--- 배정 결과 ---
  운송사 운송구간 트럭유형  운송단가  선택된트럭수
운송사02 구간01 트럭01   150       8
운송사03 구간01 트럭01   200       2
운송사02 구간01 트럭02   150       8
운송사03 구간01 트럭02   200       2
운송사02 구간02 트럭01   150       8
운송사03 구간02 트럭01   200       2
운송사02 구간02 트럭02 10000       2
운송사03 구간02 트럭02   400       8
운송사02 구간03 트럭01   400       8
운송사03 구간03 트럭01   500       2

프로그램 수행시간(초): 0.8859


4개운송사_3개운송구간_기준정보

In [3]:
import pandas as pd
import time
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger, LpBinary, value
import math

def optimize_truck_assignment():
    start_time = time.time()

    #--------------------------------------------------------
    # 1) 엑셀 데이터 읽기
    #--------------------------------------------------------
    input_file = "/content/sample_data/4개운송사_3개운송구간_기준정보.xlsx"

    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']

    max_trucker_num = pd.read_excel(input_file, sheet_name=0, usecols="M").iloc[0, 0]
    max_ratio = pd.read_excel(input_file, sheet_name=0, usecols="P").iloc[0, 0]

    # 데이터 클리닝
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)
    df_demand.dropna(subset=['필요대수'], inplace=True)

    #--------------------------------------------------------
    # 2) 파라미터 구성
    #--------------------------------------------------------
    cost_dict = {}
    capacity_dict = {}
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])
        cost_dict[key] = row['운송단가']
        capacity_dict[key] = row['트럭대수']

    demand_dict = {}
    for _, row in df_demand.iterrows():
        key = (row['운송구간'], row['트럭유형'])
        demand_dict[key] = row['필요대수']

    truckers = df['운송사'].unique()
    route_type_list = demand_dict.keys()

    #--------------------------------------------------------
    # 3) 모델 정의
    #--------------------------------------------------------
    prob = LpProblem("Truck_Optimization", LpMinimize)

    x_vars = {}
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = LpVariable(f"x_{i}_{r}_{t}", lowBound=0, cat=LpInteger)

    y_vars = {i: LpVariable(f"y_{i}", cat=LpBinary) for i in truckers}

    #--------------------------------------------------------
    # 4) 목적함수
    #--------------------------------------------------------
    prob += lpSum([cost_dict[(i, r, t)] * x_vars[(i, r, t)] for (i, r, t) in x_vars]), "Total_Transport_Cost"

    #--------------------------------------------------------
    # 5) 제약조건
    #--------------------------------------------------------

    # (1) 수요 충족
    for (r, t) in route_type_list:
        prob += lpSum([x_vars[(i, r, t)] for i in truckers if (i, r, t) in x_vars]) == demand_dict[(r, t)], f"Demand_{r}_{t}"

    # (2) 운송사별 보유 트럭수 제한
    for (i, r, t), cap in capacity_dict.items():
        prob += x_vars[(i, r, t)] <= cap, f"Capacity_{i}_{r}_{t}"

    # (3) 선택 운송사만 배정
    for i in truckers:
        for (r, t) in route_type_list:
            if (i, r, t) in x_vars:
                prob += x_vars[(i, r, t)] <= capacity_dict[(i, r, t)] * y_vars[i], f"Link_Select_{i}_{r}_{t}"

    # (4) 최대 선택 운송사 수 제한
    prob += lpSum([y_vars[i] for i in truckers]) <= max_trucker_num, "Max_Selected_Truckers"

    # (5) 트럭유형-운송구간별 운송사 배정 상한 제한
    for (r, t) in route_type_list:
        demand = demand_dict[(r, t)]
        x_max = math.ceil(demand * max_ratio)
        for i in truckers:
            if (i, r, t) in x_vars:
                prob += x_vars[(i, r, t)] <= x_max, f"MaxRatio_{i}_{r}_{t}"

    #--------------------------------------------------------
    # 6) 최적화 수행
    #--------------------------------------------------------
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    status = pulp.LpStatus[prob.status]

    print("Status:", status)
    if prob.status not in [1, 2]:
        print("유효한 해를 찾지 못했습니다.")
        return

    total_cost = value(prob.objective)
    print("총 운송비:", total_cost)

    #--------------------------------------------------------
    # 7) 결과 저장
    #--------------------------------------------------------
    result_data = []
    for (i, r, t), var in x_vars.items():
        if var.varValue > 0:
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(var.varValue)])

    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    df_result.sort_values(by=['운송구간', '트럭유형'], inplace=True)

    output_file = "/content/sample_data/4개운송사_3개운송구간_최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))

    print("\n프로그램 수행시간(초):", round(time.time() - start_time, 4))


if __name__ == "__main__":
    optimize_truck_assignment()



Status: Infeasible
유효한 해를 찾지 못했습니다.


In [4]:
import pandas as pd
import time
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger, LpBinary, value
import math

def optimize_truck_assignment():
    start_time = time.time()

    #--------------------------------------------------------
    # 1) 엑셀 데이터 읽기
    #--------------------------------------------------------
    input_file = "/content/sample_data/1개운송사만선정_기준정보.xlsx"

    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']

    max_trucker_num = pd.read_excel(input_file, sheet_name=0, usecols="M").iloc[0, 0]
    max_ratio = pd.read_excel(input_file, sheet_name=0, usecols="P").iloc[0, 0]

    # 데이터 클리닝
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)
    df_demand.dropna(subset=['필요대수'], inplace=True)

    #--------------------------------------------------------
    # 2) 파라미터 구성
    #--------------------------------------------------------
    cost_dict = {}
    capacity_dict = {}
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])
        cost_dict[key] = row['운송단가']
        capacity_dict[key] = row['트럭대수']

    demand_dict = {}
    for _, row in df_demand.iterrows():
        key = (row['운송구간'], row['트럭유형'])
        demand_dict[key] = row['필요대수']

    truckers = df['운송사'].unique()
    route_type_list = demand_dict.keys()

    #--------------------------------------------------------
    # 3) 모델 정의
    #--------------------------------------------------------
    prob = LpProblem("Truck_Optimization", LpMinimize)

    x_vars = {}
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = LpVariable(f"x_{i}_{r}_{t}", lowBound=0, cat=LpInteger)

    y_vars = {i: LpVariable(f"y_{i}", cat=LpBinary) for i in truckers}

    #--------------------------------------------------------
    # 4) 목적함수
    #--------------------------------------------------------
    prob += lpSum([cost_dict[(i, r, t)] * x_vars[(i, r, t)] for (i, r, t) in x_vars]), "Total_Transport_Cost"

    #--------------------------------------------------------
    # 5) 제약조건
    #--------------------------------------------------------

    # (1) 수요 충족
    for (r, t) in route_type_list:
        prob += lpSum([x_vars[(i, r, t)] for i in truckers if (i, r, t) in x_vars]) == demand_dict[(r, t)], f"Demand_{r}_{t}"

    # (2) 운송사별 보유 트럭수 제한
    for (i, r, t), cap in capacity_dict.items():
        prob += x_vars[(i, r, t)] <= cap, f"Capacity_{i}_{r}_{t}"

    # (3) 선택 운송사만 배정
    for i in truckers:
        for (r, t) in route_type_list:
            if (i, r, t) in x_vars:
                prob += x_vars[(i, r, t)] <= capacity_dict[(i, r, t)] * y_vars[i], f"Link_Select_{i}_{r}_{t}"

    # (4) 최대 선택 운송사 수 제한
    prob += lpSum([y_vars[i] for i in truckers]) <= max_trucker_num, "Max_Selected_Truckers"

    # (5) 트럭유형-운송구간별 운송사 배정 상한 제한
    for (r, t) in route_type_list:
        demand = demand_dict[(r, t)]
        x_max = math.ceil(demand * max_ratio)
        for i in truckers:
            if (i, r, t) in x_vars:
                prob += x_vars[(i, r, t)] <= x_max, f"MaxRatio_{i}_{r}_{t}"

    #--------------------------------------------------------
    # 6) 최적화 수행
    #--------------------------------------------------------
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    status = pulp.LpStatus[prob.status]

    print("Status:", status)
    if prob.status not in [1, 2]:
        print("유효한 해를 찾지 못했습니다.")
        return

    total_cost = value(prob.objective)
    print("총 운송비:", total_cost)

    #--------------------------------------------------------
    # 7) 결과 저장
    #--------------------------------------------------------
    result_data = []
    for (i, r, t), var in x_vars.items():
        if var.varValue > 0:
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(var.varValue)])

    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    df_result.sort_values(by=['운송구간', '트럭유형'], inplace=True)

    output_file = "/content/sample_data/1개운송사만선정_최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))

    print("\n프로그램 수행시간(초):", round(time.time() - start_time, 4))


if __name__ == "__main__":
    optimize_truck_assignment()



Status: Optimal
총 운송비: 108500.0

--- 배정 결과 ---
  운송사 운송구간 트럭유형  운송단가  선택된트럭수
운송사02 구간01 트럭01   150      10
운송사02 구간01 트럭02   150      10
운송사02 구간02 트럭01   150      10
운송사02 구간02 트럭02 10000      10
운송사02 구간03 트럭01   400      10

프로그램 수행시간(초): 0.1656
